# Simulation
This notebook will be used to import available S-parameter data from the BFP640 and simulate/verify the LNA behaviour using scikit-RF.
The simulation was done after construction, since only then I realized that LTSpice was really not up to this kind-of job.

### DC Operating point

In [1]:
Vcc = 4.8

### Checking DC Operating point (U7)
DC_VBase_U7 = 0.88			#! SIM VALUE: 1.15 V
DC_VEmitter_U7 = 0.52		#! SIM VALUE: 0.36 V
DC_VCollector_U7 = 1.84		#! SIM VALUE: 1.54 V
DC_IEmitter_U7 = DC_VEmitter_U7 / 47

### Checking DC Operating point (U6)
DC_VBase_U6 = 2.08			#! SIM VALUE: 2.33 V
DC_VEmitter_U6 = 1.84		#! SIM VALUE: 1.54 V
DC_VCollector_U6 = 2.6		#! SIM VALUE: 1.7 V
DC_IEmitter_U6 = DC_IEmitter_U7 # Assumption that Ic = Ie

### Checking DC Operating point (U8)
DC_VBase_U8 = 2.6			#! SIM VALUE: 3.45 V
DC_VEmitter_U8 = 2.32		#! SIM VALUE: 2.64 V
DC_VCollector_U8 = 4.88		#! SIM VALUE: 5 V
DC_IEmitter_U8 = DC_VEmitter_U8 / 200

print(f"VBE - U7 {DC_VBase_U7-DC_VEmitter_U7:.2f} V, U6 {DC_VBase_U6-DC_VEmitter_U6:.2f} V, U8 {DC_VBase_U8-DC_VEmitter_U8:.2f} V")
print(f"VCE - U7 {DC_VCollector_U7-DC_VEmitter_U7:.2f} V, U6 {DC_VCollector_U6-DC_VEmitter_U6:.2f} V, U8 {DC_VCollector_U8-DC_VEmitter_U8:.2f} V")
print(f"IE  - U7 {DC_IEmitter_U7*1000:.2f} mA, U6 {DC_IEmitter_U6*1000:.2f} mA, U8 {DC_IEmitter_U8*1000:.2f} mA")

VBE - U7 0.36 V, U6 0.24 V, U8 0.28 V
VCE - U7 1.32 V, U6 0.76 V, U8 2.56 V
IE  - U7 11.06 mA, U6 11.06 mA, U8 11.60 mA


In [10]:
import os
import re
import skrf as rf

# Directory containing your S2P files. Adjust as needed.
DATA_DIR = '/home/iwolfs/Work/Projects/rf_lna_project/other/s_params/SPAR/BFP640'
print(f"list directory: {os.listdir(DATA_DIR)}")
# Regex pattern to parse filenames like:
#  "BFP640_VCE_1.0V_IC_18mA.s2p"
#  "BFP640_w_noise_VCE_3.5V_IC_20mA.s2p"
# Explanation:
#   ^BFP640          match "BFP640" at start
#   (_w_noise)?      optionally match "_w_noise"
#   _VCE_           match literal
#   ([\d\.]+)V      capture group for VCE (one or more digits or '.'), ends with 'V'
#   _IC_            match literal
#   ([\d\.]+)mA     capture group for IC (similarly digits or '.')
#   \.s2p$          end with ".s2p"
pattern = re.compile(
    r'^BFP640(_w_noise)?_VCE_([\d\.]+)V_IC_([\d\.]+)mA\.s2p$',
    re.IGNORECASE
)

# Dictionary to store the data
# We'll use a tuple key: (VCE_float, IC_float, noise_bool)
# Each value is the corresponding rf.Network
networks = {}

# Iterate over all files in the directory
for fname in os.listdir(DATA_DIR):
    # Only consider .s2p files
    if not fname.lower().endswith('.s2p'):
        continue

    match = pattern.match(fname)
    if match:
        noise_str, vce_str, ic_str = match.groups()
        # Convert text to numeric
        vce_val = float(vce_str)
        ic_val = float(ic_str)
        # noise_bool = (noise_str is not None)  # True if "_w_noise" was present

        # Load the network
        full_path = os.path.join(DATA_DIR, fname)
        ntwk = rf.Network(full_path)

        # Store it in the dictionary
        # Key: (VCE, IC, noise_flag)
        # networks[(vce_val, ic_val, noise_bool)] = ntwk
        networks[(vce_val, ic_val)] = ntwk

print(f"Loaded {len(networks)} s2p files into the dictionary.")

list directory: ['BFP640_w_noise_VCE_1.0V_IC_40mA.s2p', 'BFP640_w_noise_VCE_3.0V_IC_12mA.s2p', 'BFP640_w_noise_VCE_3.5V_IC_30mA.s2p', 'BFP640_VCE_3.0V_IC_48mA.s2p', 'BFP640_w_noise_VCE_1.0V_IC_20mA.s2p', 'BFP640_w_noise_VCE_4.0V_IC_6.0mA.s2p', 'BFP640_VCE_1.0V_IC_32mA.s2p', 'BFP640_w_noise_VCE_2.5V_IC_8.0mA.s2p', 'BFP640_w_noise_VCE_1.0V_IC_5.0mA.s2p', 'BFP640_w_noise_VCE_3.0V_IC_15mA.s2p', 'BFP640_VCE_3.0V_IC_18mA.s2p', 'BFP640_VCE_2.5V_IC_28mA.s2p', 'BFP640_w_noise_VCE_2.0V_IC_40mA.s2p', 'BFP640_VCE_1.0V_IC_37mA.s2p', 'BFP640_w_noise_VCE_4.0V_IC_35mA.s2p', 'BFP640_w_noise_VCE_2.5V_IC_2.0mA.s2p', 'BFP640_w_noise_VCE_3.0V_IC_2.0mA.s2p', 'BFP640_VCE_3.5V_IC_50mA.s2p', 'BFP640_VCE_1.5V_IC_18mA.s2p', 'BFP640_w_noise_VCE_4.0V_IC_1.0mA.s2p', 'BFP640_w_noise_VCE_3.5V_IC_45mA.s2p', 'BFP640_w_noise_VCE_2.5V_IC_10mA.s2p', 'BFP640_VCE_3.5V_IC_18mA.s2p', 'BFP640_w_noise_VCE_4.0V_IC_4.0mA.s2p', 'BFP640_VCE_2.5V_IC_32mA.s2p', 'BFP640_w_noise_VCE_1.0V_IC_15mA.s2p', 'BFP640_w_noise_VCE_1.5V_IC_6.0mA.

### Circuit design

In [ ]:
import skrf as rf
import numpy as np
from skrf.media import MLine, CPW, DefinedGammaZ0
from skrf.circuit import Circuit, Circuit_3port, Network
Z0 = 50
# Create line for microstrip lines and components
line = DefinedGammaZ0(freq, z0=Z0)

# Create passive components
# Inductors
L5 = line.inductor(160e-9, name='L5')  # 160nH
L6 = line.inductor(160e-9, name='L6')  # 160nH
L13 = line.inductor(160e-9, name='L13')  # 160nH
L12 = line.inductor(160e-9, name='L12')  # 160nH
L9 = line.inductor(160e-9, name='L9')   # 160nH

# Capacitors
C17 = line.capacitor(100e-12, name='C17')  # 100pF
C7 = line.capacitor(100e-12, name='C7')   # 100pF

# Resistors
R10 = line.resistor(2.2e3, name='R10')  # 2.2kΩ
R11 = line.resistor(1e3, name='R11')    # 1kΩ
R12 = line.resistor(1e3, name='R12')    # 1kΩ
R13 = line.resistor(200, name='R13')    # 200Ω
R14 = line.resistor(0, name='R14')      # 0Ω
R15 = line.resistor(47, name='R15')     # 47Ω
R16 = line.resistor(200, name='R16')    # 200Ω

# Impedances/Transmission lines
Z1 = line.short(name='Z1').add_z0_at_ports([3.6])  # 3.6pF transmission line
Z2 = line.short(name='Z2').add_z0_at_ports([1])    # 1pF transmission line
Z3 = line.open(name='Z3')                          # DNP (Do Not Place)
Z4 = line.short(name='Z4').add_z0_at_ports([3.6])  # 3.6pF transmission line
Z5 = line.short(name='Z5').add_z0_at_ports([3.6])  # 3.6pF transmission line
Z6 = line.inductor(15e-9, name='Z6')               # 15nH (DNP)
Z7 = line.open(name='Z7')                          # DNP
Z20 = line.short(name='Z20').add_z0_at_ports([20]) # 20nH transmission line

# Create placeholder transistors
U6 = rf.Circuit('U6_BFP640', freq)
U7 = rf.Circuit('U7_BFP640', freq)
U8 = rf.Circuit('U8_BFP640', freq)